# Comparing McKinley Park's PurpleAir (PA) sensors to EPA AirNow's (AN) sensors using accuracy correction formulae

In this notebook we recreate the analysis from the `compare_airnow_and_purpleair` notebook but we take into account various correction formulae for the PA sensors found in the literature.

## TODO Literature review

In this section we briefly review some of the literature that compares the PM2.5 measurements of the PurpleAir sensors to that of other sensors.

###  Basic notions

### Comparisons and correction formulae

### References

[1] https://cfpub.epa.gov/si/si_public_record_Report.cfm?dirEntryId=350075&Lab=CEMM

[2] https://www.sciencedirect.com/science/article/abs/pii/S135223102100251X

[3] https://api.purpleair.com/

[4] https://map.purpleair.com

[5] http://lar.wsu.edu/nw-airquest/docs/20200610_meeting/NWAQ_20200611_1030_Hadley.pdf

## Analysis

### Imports and utility

In [83]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.display.precision = 1

PATH = '../data/'
SEASONS = ['summer', 'autumn']
FILE_STUBS = {'an': '_2021_airnow',
              'pa': '_2021_pa_'}
CHANNELS = {'a': 'parent',
            'b': 'child'}

# features of interest (and how we'll rename them)
COLUMNS = {'pa': {'season': 'season',
                  'sensor': 'sensor',
                  'created_at': 'date',
                  'pm2.5_(cf=1)_ug/m3': 'pm25_cf_1',
                  'pm2.5_(cf=atm)_ug/m3': 'pm25_cf_atm',
                  'temperature_f': 'temperature_f',
                  'humidity_%': 'humidity_%'},
           'an': {'season': 'season',
                  'latitude': 'latitude',
                  'longitude': 'longitude',
                  'datetime': 'date',
                  'concentration': 'pm25'}}

### Load data into PA and AN DataFrames

Each DataFrame contains data for PM2.5, humidity, etc. for both seasons. Following [1], the PurpleAir data is averaged over the parent and child data (A and B channels) when possible. From the PA sensors, we keep only those located in McKinley Park, which is our region of interest.

#### TODO what happens when the A and B channels differ significantly?

In [84]:
def load_data(dataset):
    # TODO documentation
    
    df = pd.DataFrame()
    
    if dataset == 'pa':
        for season in SEASONS:
            for channel in CHANNELS.keys():
                filename = f'{PATH}{season}{FILE_STUBS[dataset]}{CHANNELS[channel]}.csv'
                df_season_channel = pd.read_csv(filename)
                df_season_channel['season'] = season
                df = df.append(df_season_channel)
        
    elif dataset == 'an':
        for season in SEASONS:
            filename = f'{PATH}{season}{FILE_STUBS[dataset]}.csv'
            df_season = pd.read_csv(filename)
            # keep only PM2.5 data
            df_season = df_season[df_season['Parameter'] == 'PM2.5']
            df_season['season'] = season
            df = df.append(df_season)
        
    else:
        raise ValueError(f'Could not find dataset {dataset}!')
        
    # clean up headers, prune extraneous data
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df = df[list(COLUMNS[dataset].keys())]
    df.rename(columns = COLUMNS[dataset], inplace = True)
    
    # convert date format to 'mm-yy HH:mm' (24hour time)
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.strftime('%m/%d %H:%M')
    
    # for PurpleAir data, average the PM2.5 readings between the a and b channels
    # and keep the humidity and temperature readings, which are only present in the a channel
    
    return df

For the PurpleAir data, we'll average the PM2.5 readings between the a and b channels (throwing away any points for which one is missing). We'll use the PM2.5 CF=1 data, and moreover, keep the temperature and humidity data, which is only present on the a channel.

The a channel and b channel have sensor IDs that differ by 1.

In [90]:
df_pa = load_data('pa')

SENSOR_PAIRS = {'36th and Paulina': (96035, 96036),
                '38th & Winchester': (96395, 96396),
                '39th and Damen (NLEI)': (94975, 94976)}

df_pa_ab_avg = pd.DataFrame()
for monitor in SENSOR_PAIRS.keys():
    df_a = df_pa[df_pa['sensor'] == SENSOR_PAIRS[monitor][0]]
    df_b = df_pa[df_pa['sensor'] == SENSOR_PAIRS[monitor][1]][['date', 'pm25_cf_1']]
    
    df_merged = df_a.merge(df_b, left_on = 'date', right_on = 'date')
    df_merged['pm25'] = (df_merged['pm25_cf_1_x'] + df_merged['pm25_cf_1_y']) / 2
    df_merged['sensor'] = monitor
    df_merged = df_merged[['season', 'sensor', 'date', 'pm25', 'temperature_f', 'humidity_%']]
    
    df_pa_ab_avg = df_pa_ab_avg.append(df_merged)

df_pa_ab_avg.head()

,season,sensor,date,pm25,temperature_f,humidity_%
0,summer,36th and Paulina,07/25 00:01,15.3,87.0,65.0
1,summer,36th and Paulina,07/25 00:03,13.7,87.0,66.0
2,summer,36th and Paulina,07/25 00:05,14.8,86.0,67.0
3,summer,36th and Paulina,07/25 00:07,17.7,87.0,67.0
4,summer,36th and Paulina,07/25 00:09,18.4,87.0,66.0


There's not as much to do for the AirNow data. We'll just replace the latitude/longitude with a human-readable location.

In [99]:
df_an = load_data('an')
sensorCoords_an = df_an[['latitude', 'longitude']].drop_duplicates().sort_values(by='latitude').reset_index()
locations_an = {0: 'Alsip',
                1: 'Springfield Pump',
                2: 'Cicero',
                3: 'Com Ed',
                4: 'Des Plaines'}               
sensorCoords_an['sensor'] = locations_an.values()
df_an = df_an.merge(sensorCoords_an[['latitude', 'longitude', 'sensor']],
                    how='outer', on=['latitude', 'longitude'])
df_an = df_an.drop(['latitude', 'longitude'], axis = 1)
df_an.head()

,season,date,pm25,sensor
0,summer,07/04 02:00,10.5,Des Plaines
1,summer,07/04 03:00,12.2,Des Plaines
2,summer,07/04 04:00,16.1,Des Plaines
3,summer,07/04 05:00,18.1,Des Plaines
4,summer,07/04 06:00,16.6,Des Plaines


### Take daily averages, plot the correlation, compare

### Apply accuracy correction formulae, compare